# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Write functions to parse specific information from a web page
* Iterate over successive web pages in order to create a dataset

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [1]:
from bs4 import BeautifulSoup
import requests

In [43]:
#html_page = requests.get('http://books.toscrape.com/') 

html_page = requests.get('http://books.toscrape.com/catalogue/page-1.html')
soup = BeautifulSoup(html_page.content, 'html.parser') 

In [4]:
#soup.prettify()

In [44]:
def retrieve_titles(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    my_titles = book_container.findAll('img')
    titles = [title['alt'] for title in my_titles]
    return titles

In [45]:
titles = retrieve_titles(soup)

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given html page and return a list of the star-ratings for the books. These star ratings should be formatted as integers.

In [5]:
import re
regex = re.compile("star-rating (.*)")
ratings = soup.findAll('p', {"class" : regex})

In [8]:
ratings[0].attrs

{'class': ['star-rating', 'Three']}

In [6]:
def retrieve_ratings(soup):
    regex = re.compile("star-rating (.*)")
    ratings = soup.findAll('p', {"class" : regex})
    ratings_list = [rating['class'][1] for rating in ratings]
    
    star_dict = {'One': 1, 'Two': 2, 'Three':3, 'Four': 4, 'Five':5} #Manually create a dictionary to translate to numeric
    star_ratings = [star_dict[s] for s in ratings_list]
    return ratings_list

In [7]:
ratings_list = retrieve_ratings(soup)

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [13]:
def retrieve_prices(soup):
    prices = [p.text for p in soup.findAll('p', class_="price_color")]
    prices = [float(p[1:]) for p in prices]
    return prices

In [14]:
prices = retrieve_prices(soup)

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [38]:
import re
def retrieve_availabilities(soup):
    PAT = r'(\s*)(\w+\s*\w*)(\s*)'
    avail_list = []
    avails = soup.findAll('p', class_="instock availability")
    for item in avails:
        m = re.search(PAT, item.text)
        avail_list.append(m[2])
    
    return avail_list


In [36]:
avail_list = retrieve_availabilities(soup)

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [39]:
BASE_URL = 'http://books.toscrape.com/catalogue/page-{}.html'

In [40]:
page = 1

In [41]:
url = BASE_URL.format(page)

In [42]:
url

'http://books.toscrape.com/catalogue/page-1.html'

In [47]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

BASE_URL = 'http://books.toscrape.com/catalogue/page-{}.html'
TOTAL = 50
all_titles_df = pd.DataFrame()
all_titles_df.columns = ['Title', 'Star_Rating', 'Price_(pounds)', 'Availability']

# static iteration over "TOTAL" number of web pages
for page in range(1, TOTAL):
    url = BASE_URL.format(page)
    html_page = requests.get(url) 
    soup = BeautifulSoup(html_page.content, 'html.parser') 
    
    title_list = retrieve_titles(soup)
    ratings_list = retrieve_ratings(soup)
    prices_list = retrieve_prices(soup)
    avail_list = retrieve_availabilities(soup)
    
    df = pd.DataFrame([titles_list, ratings_list, prices_list, avail_list]).transpose()
    all_titles_df = pd.concat([all_titles_df, df], ignore_index=True)
    

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

## Level-Up: Write a new version of the script you just wrote. 

If you used url hacking to generate each successive page url, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [ ]:
#Your code here

## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!